In [ ]:
#PACKAGES
import requests
import json
import pandas

In [ ]:
#SOCIB API settings
api_key = '' #write you SOCIB api_key => get it from http://api.socib.es/home/ (formulary at the bottom of the page)
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

In [ ]:
#API QUERY ELEMENTS
platform = 'Mobims_SonBou' #check for more ids at: http://api.socib.es/platforms/ ---> filter by passing a platform_type parameter if needed (see available codes at: http://api.socib.es/platform-types/)
end_point = '/entries/?platform='+platform

In [ ]:
#Getting raw array of netCDFs (entries) from a given platform -> the resulting array preserves defauting fields 
apiCache = {}
entries = []
url = '%s%s' % (api_url, end_point)
while url != None:
    request_ = requests.get(url, headers=headers)
    response_ = json.loads(request_.text)
    for entry in response_['results']:#loop over netCDFs (entries)
        data = {}
        data['id'] = entry['id']
        data['netCDF_name'] = entry['services']['opendap']['url'].split('/')[-1]
        data['processing_level'] = entry['processing_level']
        data['data_mode'] = entry['data_mode']
        data['initial_datetime'] = entry['initial_datetime']
        data['end_datetime'] = entry['end_datetime']
        data['variables'] = entry['variables']
        data['opendap_url'] = entry['services']['opendap']['url']
        data['http_url'] = entry['services']['http_file']['url']
        data['catalog_url'] = entry['services']['thredds_catalog']['url']
        if entry['instrument']!=None:
            #instrument name
            url = entry['instrument'].replace('data-','')
            try:
                response = apiCache[url]
            except Exception as e:
                request = requests.get(url, headers=headers)
                apiCache[url] = json.loads(request.text)
                response = apiCache[url]
            data['instrument_name'] = response['name']
            #instrument type and category
            url = response['type']
            try:
                response = apiCache[url]
            except Exception as e:
                try:
                    request = requests.get(url, headers=headers)
                    apiCache[url] = json.loads(request.text)
                    response = apiCache[url]
                except:
                    response['name'] = ''
                    response['type'] = ''
            data['instrument_type'] = response['name']
            data['instrument_category'] = response['type']
        else:
            data['instrument_name'] = ''
            data['instrument_type'] = ''
            data['instrument_category'] = ''
        entries.append(data)
    url = response_['next']
    print(url)

In [ ]:
#Getting CSV -> ordering the ouput by date, isntrument type, instrument name and processing level
csv_name = platform+'_netCDFs.csv'
ordered_header = ['instrument_category', 'instrument_type', 'instrument_name', 'processing_level', 'data_mode', 'initial_datetime', 'end_datetime', 'variables', 'netCDF_name', 'id', 'opendap_url', 'http_url', 'catalog_url']
nc_dataframe = pandas.DataFrame.from_dict(entries).sort_values(['instrument_category','instrument_type', 'processing_level', 'instrument_name', 'end_datetime'])
nc_dataframe[ordered_header].to_csv(csv_name,index=False)